## Agent-Based Code Execution using Amazon AgentCore Bedrock Code Interpreter - Tutorial(Langchain)
This tutorial demonstrates how to create an AI agent that validates answers through code execution using Python. We use Amazon Bedrock AgentCore Code Interpreter to run code that is generated by the LLM

This tutorial demonstrates how to use AgentCore Bedrock Code Interpreter to:
1. Set up a sandbox environment
2. Configure a langchain based agent that generated code based on the user query
3. Execute code in a sandbox environment using Code Interpreter
4. Display the results back to the user

## Prerequisites
- AWS account with Bedrock AgentCore Code Interpreter access
- You have the necessary IAM permissions to create and manage code interpreter resources
- Required Python packages installed(including boto3, bedrock-agentcore & langchain)
- IAM role should have permissions to invoke models on Amazon Bedrock
 - Access to Claude 3.5 Sonnet model in the US Oregon (us-west-2) region

## Your IAM execution role should have the following IAM policy attached



~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## How it works

The code execution sandbox enables agents to safely process user queries by creating an isolated environment with a code interpreter, shell, and file system. After a Large Language Model helps with tool selection, code is executed within this session, before being returned to the user or Agent for synthesis.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries and initialize our Code Interpreter session.

In [ ]:
!pip install --upgrade -r requirements.txt

In [17]:
from bedrock_agentcore.tools.code_interpreter_client import code_session
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, tool
from langchain_aws import ChatBedrockConverse
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import json


## 2.System Prompt Definition
Define the behavior and capabilities of the AI assistant. We instruct our assistant to always validate answers through code execution and data based reasoning.

In [8]:
SYSTEM_PROMPT = """You are a helpful AI assistant that validates all answers through code execution.

VALIDATION PRINCIPLES:
1. When making claims about code, algorithms, or calculations - write code to verify them
2. Use execute_python to test mathematical calculations, algorithms, and logic
3. Create test scripts to validate your understanding before giving answers
4. Always show your work with actual code execution
5. If uncertain, explicitly state limitations and validate what you can

APPROACH:
- If asked about a programming concept, implement it in code to demonstrate
- If asked for calculations, compute them programmatically AND show the code
- If implementing algorithms, include test cases to prove correctness
- Document your validation process for transparency
- The sandbox maintains state between executions, so you can refer to previous results

TOOL AVAILABLE:
- execute_python: Run Python code and see output

RESPONSE FORMAT: The execute_python tool returns a JSON response with:
- sessionId: The sandbox session ID
- id: Request ID
- isError: Boolean indicating if there was an error
- content: Array of content objects with type and text/data
- structuredContent: For code execution, includes stdout, stderr, exitCode, executionTime"""

## 3.Code Execution Tool Definition
Next we define the function as tool that will be used by the Agent as tool, to run code in the code sandbox. We use the @tool decorator to annotate the function as a custom tool for the agent.

Within an active code interpreter session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [11]:
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""
    
    if description:
        code = f"# {description}\n{code}"
    
    print(f"\n Generated Code: {code}")
    
    with code_session("us-west-2") as code_client:
        response = code_client.invoke("executeCode", {
            "code": code,
            "language": "python",
            "clearContext": False
        })
    
    for event in response["stream"]:
        return json.dumps(event["result"])

## 4. Agent Configuration
We create and configure an agent using the langchain SDK. We provide it the system prompt and the tool, we defined above to execute generate code

#### 4.1 Initialize the language model

In [6]:
llm = ChatBedrockConverse(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",region_name="us-west-2")

#### 4.2 Define the prompt template

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

#### 4.3 Create a list of our custom tools

In [ ]:
tools = [execute_python]

### 4.4 Create the agent executor


In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 5. Query Definition
Define a sample query to test the agent with code execution capabilities

In [ ]:
query="Can all the planets in the solar system fit between the earth and moon?"

## 6. Agent Invocation and Response Processing
We invoke the agent with our query and process the agent's response, Note that the agent performs reasoning by starting with a hypothesis, validates by converting it into code and executes the code in the code interpreter

In [ ]:
response=agent_executor.invoke({"input": query})

### The final response from our agent...

In [ ]:
print(response['output'][0]['text'])